# Introduction

Preprocesses NASA VNP46A1 HDF5 files. This Notebook takes raw `.h5` files and completes the following preprocessing tasks:

* Extracts radiance and qualify flag bands;
* Masks radiance for fill values, clouds, and sensor problems;
* Fills masked data with NaN values;
* Creates a georeferencing transform;
* Creates export metadata; and,
* Exports radiance data to GeoTiff format.

This Notebook uses the following folder structure:

* `nighttime-radiance/`
    * `01-code-scripts/`
        * `preprocess_vnp46a1.py`
    * `02-raw-data/`
    * `03-processed-data/`
    * `04-graphics-outputs/`
    * `05-papers-writings/`

Running the Notebook from the `01-code-scripts/` folder works by default. If the Notebook runs from a different folder, the paths in the environment setup section may have to be changed.

# Environment Setup

In [ ]:
# Load Notebook formatter
%load_ext nb_black
# %reload_ext nb_black

In [ ]:
# Import packages
import os
import warnings
import glob
import viirs

In [ ]:
# Set options
warnings.simplefilter("ignore")

In [ ]:
# Set working directory
os.chdir("..")

# User-Defined Variables

In [ ]:
# Define path folder containing input VNP46A1 HDF5 files
hdf5_input_folder = os.path.join("02-raw-data", "hdf", "south-korea")

# Defne path to output folder to store exported GeoTiff files
geotiff_output_folder = os.path.join(
    "03-processed-data", "raster", "south-korea", "vnp46a1-grid"
)

# Data Preprocessing

In [ ]:
# Preprocess each HDF5 file (extract bands, mask for fill values, clouds, and
#  sensor problems, fill masked values with NaN, export to GeoTiff)
hdf5_files = glob.glob(os.path.join(hdf5_input_folder, "*.h5"))
processed_files = 0
total_files = len(hdf5_files)
for hdf5 in hdf5_files:
    viirs.preprocess_vnp46a1(
        hdf5_path=hdf5, output_folder=geotiff_output_folder
    )
    processed_files += 1
    print(f"Preprocessed file: {processed_files} of {total_files}\n\n")

Workflow:
* Download HDF files
* Load HDF files
* Read radiance band into array
* Read cloud mask data into array 
* Read other QA data into array
* Create stack of bands to use
* Clip stack
* Mask for DNB band quality
* Mask for clouds
* Mask radiance based on QA/cloud mask flags
* Set no data/masked to 0
* Set data outside bounds to 0
* Store in dictionary based on year/month/day/time
* Get metadata
* Export radiance to GeoTiff



# Notes and References

Links:

* https://ladsweb.modaps.eosdis.nasa.gov/missions-and-measurements/products/VNP46A1/
* https://viirsland.gsfc.nasa.gov/PDF/VIIRS_BlackMarble_UserGuide.pdf

Download HDF5 files taken care of by `download_laads_order.py`

File naming convention:

VNP46A1.AYYYYDDD.hXXvYY.CCC.YYYYDDDHHMMSS.h5

* VNP46A1 = Short-name
* AYYYYDDD = Acquisition Year and Day of Year
* hXXvYY = Tile Identifier (horizontalXXverticalYY)
* CCC = Collection Version
* YYYYDDDHHMMSS = Production Date – Year, Day, Hour, Minute, Second
* h5 = Data Format (HDF5)

Bands of interest (table from pages 12-13 of handbook)

| Scientific Dataset          | Units             | Description            | Bit Types               | Fill Value | Valid Range | Scale Factor | Offset |
|:-----------------------------|:-------------------|:------------------------|:-------------------------|:------------|:-------------|:--------------|:--------|
| DNB_At_Sensor_Radiance_500m | nW_per_cm2_per_sr | At-sensor DNB radiance | 16-bit unsigned integer | 65535      | 0 - 65534   | 0.1          | 0.0    |
| QF_Cloud_Mask               | Unitless          | Cloud mask status      | 16-bit unsigned integer | 65535      | 0 - 65534   | N/A          | N/A    |
| QF_DNB                      | Unitless          | DNB_quality flag       | 16-bit unsigned integer | 65535      | 0 - 65534   | N/A          | N/A    |
| UTC_Time                    | Decimal hours     | UTC Time               | 32-bit floating point   | -999.9     | 0 - 24      | 1.0          | 0.0    |

How to get Georeferencing info not using GDAL? extract bounds from file using netCDF4 package?

NASA Scripts:

https://git.earthdata.nasa.gov/projects/LPDUR/repos/nasa-viirs/browse/scripts

https://git.earthdata.nasa.gov/projects/LPDUR/repos/nasa-viirs/browse/scripts/VIIRS_HDF5toGeoTIFF.py

https://urs.earthdata.nasa.gov/

Workflow:

* Mask stack for no data
* Mask stack for QA (cloud, snow, shadow, etc.)
* Scale stack
* Fill stack
    * Have different fill values for different layers?
    * And/or have fill values that are of the same `dtype` as the original layer?

Masking Criteria:

* mask where DNB_At_Sensor_Radiance_500m == 65535
* mask where DNB_At_Sensor_Radiance_500m > 65534
* mask where DNB_At_Sensor_Radiance_500m < 0
* mask where QF_Cloud_Mask == 2 (Probably Cloudy)
* mask where QF_Cloud_Mask == 3 (Confident Cloudy)
* mask where QF_DNB != 0 (0 = no problems, any other number means some kind of issue)

* Extract bands
* Mask for fill values
* Mask for clouds
* Mask for sensor problems
* Fill masked values
* Create transform
* Create metadata
* Export array to GeoTiff

* QF_Cloud_Mask
    * Bit 0 - Day/Night 	
        * 0 (Night)
        * 1 (Day)
        
    * Bits 1-3 - Land/Water Background
        * 0 (Land & Desert)
        * 1 (Land no Desert)
        * 2 (Inland Water)
        * 3 (Sea Water)
        * 5 (Coastal)
        
    * Bits 4-5 - Cloud Mask Quality 	
        * 0 (Poor)
        * 1 (Low)
        * 2 (Medium)
        * 3 (High)
    
    * Bits 6-7 - Cloud Detection Results & Confidence Indicator 
        * 0 (Confident Clear)
        * 1 (Probably Clear)
        * 2 (Probably Cloudy)
        * 3 (Confident Cloudy)

    * Bit 8 - Shadow Detected
        * 1 (Yes)
        * 0 (No)

    * Bit 9 - Cirrus Detection (IR) (BTM15 –BTM16) 	
        * 1 (Cloud)
        * 0 (No Cloud)

    * Bit 10 - Snow/Ice Surface
        * 1 (Snow/Ice)
        * 0 (No Snow/Ice)

QF_Cloud_Mask (base-2):

| Bit | Flag Description Key                          | Interpretation                                                                            |
|:-----|:-----------------------------------------------|:-------------------------------------------------------------------------------------------|
| 0   | Day/Night                                     | 0 = Night <br> 1 = Day                                                                         |
| 1-3 | Land/Water Background                         | 000 = Land & Desert <br> 001 = Land no Desert <br> 010 = Inland Water <br> 011 = Sea Water <br> 101 = Coastal |
| 4-5 | Cloud Mask Quality                            | 00 = Poor <br> 01 = Low <br> 10 = Medium <br> 11 = High                                                  |
| 6-7 | Cloud Detection Results & Confidence Indicator | 00 = Confident Clear <br> 01 = Probably Clear <br> 10 = Probably Cloudy <br> 11 = Confident Cloudy     |
| 8   | Shadow Detected                               | 1 = Yes <br> 0 = No                                                                            |
| 9   | Cirrus Detection (IR) (BTM15 –BTM16)          | 1 = Cloud <br> 0 = No Cloud                                                                   |
| 10  | Snow/Ice Surface                              | 1 = Snow/Ice <br> 0 = No Snow/Ice                                                            |

QF_Cloud_Mask (base-10):

| Bit | Flag Description Key                          | Interpretation                                                                            |
|:-----|:-----------------------------------------------|:-------------------------------------------------------------------------------------------|
| 0   | Day/Night                                     | 0 = Night <br> 1 = Day                                                                         |
| 1-3 | Land/Water Background                         | 0 = Land & Desert <br> 1 = Land no Desert <br> 2 = Inland Water <br> 3 = Sea Water <br> 5 = Coastal |
| 4-5 | Cloud Mask Quality                            | 0 = Poor <br> 1 = Low <br> 2 = Medium <br> 3 = High                                                  |
| 6-7 | Cloud Detection Results & Confidence Indicator | 0 = Confident Clear <br> 1 = Probably Clear <br> 2 = Probably Cloudy <br> 3 = Confident Cloudy     |
| 8   | Shadow Detected                               | 1 = Yes <br> 0 = No                                                                            |
| 9   | Cirrus Detection (IR) (BTM15 –BTM16)          | 1 = Cloud <br> 0 = No Cloud                                                                   |
| 10  | Snow/Ice Surface                              | 1 = Snow/Ice <br> 0 = No Snow/Ice                                                            |

QF_DNB:

| SDS Layer | Flag Mask Values and Descriptions|
|:-----------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| QF_DNB    | 1 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; = Substitute_Cal<br>2&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; = Out_of_Range<br>4&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; = Saturation<br>8&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; = Temp_not_Nominal<br>16&nbsp;&nbsp;&nbsp;&nbsp; = Stray_Light<br>256&nbsp;&nbsp; = Bowtie_Deleted/Range_Bit<br>512&nbsp;&nbsp; = Missing_EV<br>1024 = Cal_Fail<br>2048 = Dead_Detector |

https://ladsweb.modaps.eosdis.nasa.gov/missions-and-measurements/products/VNP46A1/